In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Bespoke asset classes

Demonstration of how to create your own custom instrument inside LUSID, create a transaction against it and value it.

Attributes
----------
instruments
transactions
properties
quotes
aggregation
"""

toggle_code("Toggle Docstring")

A common frustration with investment technology platforms is that there is always an asset class in the client security universe that is unsupported and requires bespoke handling.

For most institutions, trading in new asset classes is restricted until a front-to-back technology solution is built to support them.

Fortunately, LUSID was designed with this ability in mind.

Using LUSID you can easily create bespoke assets. This is ideal for alternative investments such as complex derivatives, loan products or even purchases of real estate (which we’ll use in this example).
In this notebook you will learn how to create your own custom instrument inside LUSID, create a transaction against it to add it to a portfolio and finally value it.

*First of all, run the cell below to initialise LUSID and import the relevant libraries*

In [2]:
# Import LUSID
import lusid.models as models
import lusid
import lusid_sample_data as import_data
from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import numpy as np
import json
import uuid
import os

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print("LUSID Environment Initialised")
print(
    "LUSID SDK Version: ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
LUSID SDK Version:  0.0.1.0


![Scopes](img/paper-lusid.gif)

In LUSID you can create separate environments using scopes. A scope is a container for LUSID objects and can be thought of as a separate identity namespace. Using LUSID's entitlements engine you can ensure that a user can only interact with objects inside their scope. Furthermore no other stakeholder inside the organisation can access this scope unless explicitly given permission. 

For this example you will create a scope to hold your commercial real estate assets.

*Run the cell below to initialise your scope. Note that the scope will have a unique 4 character code appended to it to make the name more unique.*

In [3]:
scope = "Commercial-Real-Estate-" + import_data.create_scope_id()
prettyprint.heading("Scope", scope)

Scope: Commercial-Real-Estate-3b23-0554-3a58-2c


![Scopes](img/complexinstruments-scope.gif)

Now that you have a scope you can create your custom asset. In this situation you have just purchased a shopping centre in Sydney. You have some of the important details of the asset in a small JSON file. You haven't been able to enter this into your internal systems (yet) as there are no fields available for some of the more specific information such as the average retail lease. 

With LUSID you can create an instrument for this asset and attach all of this information to it without any modification.

*Run the cell below to upsert the instrument into LUSID with the attached bespoke information*

In [4]:
# Create the bespoke hedge contract definition
shopping_centre_asset = {
    "identifier": "SYD_241232",
    "no_units": 76,
    "average_retail_lease": "2.3 years",
    "term": "5yr",
    "freq": "Quarterly",
    "rating": "B1",
}

# Create the definition for your instrument, attaching the bespoke contract
shopping_centre_asset_instrument = models.InstrumentDefinition(
    name="Sydney_Bondi_Junc_Westfield",
    identifiers={
        "ClientInternal": models.InstrumentIdValue(
            value=shopping_centre_asset["identifier"]
        )
    },
    definition=models.ExoticInstrument(
        instrument_format=models.InstrumentDefinitionFormat(
            source_system="ClientSystemA", vendor="ClientA", version="0.0.1"
        ),
        content=json.dumps(shopping_centre_asset),
        instrument_type="ExoticInstrument",
    ),
)

response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    request_body={"shopping_centre": shopping_centre_asset_instrument}
)

prettyprint.instrument_response(response)

Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,shopping_centre,SYD_241232,LUID_00003D4V


![Scopes](img/complexinstruments-instrumentmaster.gif)

You can check that the instrument has been created in your instrument master inside LUSID by retrieving it. 

*Run the cell below to get the instrument from the instrument master*

In [5]:
response = api_factory.build(lusid.api.InstrumentsApi).get_instrument(
    identifier_type="ClientInternal", identifier=shopping_centre_asset["identifier"]
)

# Print the response
print(
    "Instrument Name: " + response.name + "\n",
    "ClientInternal: " + response.identifiers["ClientInternal"] + "\n",
    "Lusid Instrument ID: " + response.lusid_instrument_id + "\n",
    "Instrument Definition:" + "\n" + response.instrument_definition.content,
)

luid = response.lusid_instrument_id

Instrument Name: Sydney_Bondi_Junc_Westfield
 ClientInternal: SYD_241232
 Lusid Instrument ID: LUID_00003D4V
 Instrument Definition:
{"identifier": "SYD_241232", "no_units": 76, "average_retail_lease": "2.3 years", "term": "5yr", "freq": "Quarterly", "rating": "B1"}


Now that you've created the instrument, it would be great if you could split out some of the properties such as the average retail lease into a more structured schema so that you can access it more easily, search by it etc.

In LUSID you have complete control over the schema you'd like to set up for your instruments. In this case you can take each bit of important information from the details about the asset and create properties for them.

Unlike the unstructured instrument definition, these properties have a defined data type, and you can enforce different types of validation on them.

*Run the cell below to extract the important details from the assets definition and create them as properties with a defined data type*

In [6]:
properties = []
property_keys = {}

for key, value in shopping_centre_asset.items():

    if type(value) in ["int", "float"]:
        data_type = "number"
        property_value = models.PropertyValue(
            metric_value=models.MetricValue(value=value)
        )

    else:
        data_type = "string"
        property_value = models.PropertyValue(label_value=value)

    # Create your request to define a new property
    property_request = models.CreatePropertyDefinitionRequest(
        domain="Instrument",
        scope=scope,
        code=key,
        value_required=False,
        display_name=key,
        data_type_id=models.ResourceId(scope="system", code=data_type),
    )

    # Call LUSID to create uour new property
    response = api_factory.build(
        lusid.api.PropertyDefinitionsApi
    ).create_property_definition(create_property_definition_request=property_request)

    # Grab the key off the response to use when referencing this property in other LUSID calls
    property_key = response.key

    property_keys[key] = property_key

    # Pretty print your key
    prettyprint.heading("{} Property Key".format(key), property_key)

    properties.append(models.ModelProperty(key=property_key, value=property_value))

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(
    upsert_instrument_property_request=[
        models.UpsertInstrumentPropertyRequest(
            identifier_type="LusidInstrumentId", identifier=luid, properties=properties
        )
    ]
)

identifier Property Key: Instrument/Commercial-Real-Estate-3b23-0554-3a58-2c/identifier
no_units Property Key: Instrument/Commercial-Real-Estate-3b23-0554-3a58-2c/no_units
average_retail_lease Property Key: Instrument/Commercial-Real-Estate-3b23-0554-3a58-2c/average_retail_lease
term Property Key: Instrument/Commercial-Real-Estate-3b23-0554-3a58-2c/term
freq Property Key: Instrument/Commercial-Real-Estate-3b23-0554-3a58-2c/freq
rating Property Key: Instrument/Commercial-Real-Estate-3b23-0554-3a58-2c/rating


Now that you've created these properties, they can be used for similar instruments. You can pull the relevant properties off each instrument with ease now. For example perhaps you are only interested in the rating and the term of this particular piece of commerial real estate.

*Run the cell below to pick off the relevant properties for your instrument*

In [7]:
response = api_factory.build(lusid.api.InstrumentsApi).get_instrument(
    identifier_type="ClientInternal",
    identifier=shopping_centre_asset["identifier"],
    property_keys=[property_keys["rating"], property_keys["term"]],
)

# Print the response
print(
    "Instrument Name: " + response.name + "\n",
    "ClientInternal ID: " + response.identifiers["ClientInternal"] + "\n",
    "Lusid Instrument ID: " + response.lusid_instrument_id,
)

for property in response.properties:
    print(" {}: ".format(property.key.split("/")[2]) + property.value.label_value)

Instrument Name: Sydney_Bondi_Junc_Westfield
 ClientInternal ID: SYD_241232
 Lusid Instrument ID: LUID_00003D4V
 term: 5yr
 rating: B1


Now that you've created the instrument with its entire definition attached, as well as creating custom properties to more easily access the important information, you can hold this instrument in a portfolio.

In this case let's create a new portolio to hold the asset.

*Run the cell below to create a portfolio*

In [8]:
portfolio_code = str(uuid.uuid4())

# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3)).isoformat()

# Create the request to add our portfolio
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name="CommercialRealEstate",
    code=portfolio_code,
    base_currency="AUD",
    description="Commercial Real Estate Assets including Shopping Centres",
    created=portfolio_creation_date,
)

# Call LUSID to create our portfolio
portfolio_response = api_factory.build(
    lusid.api.TransactionPortfoliosApi
).create_portfolio(
    scope=scope, create_transaction_portfolio_request=transaction_portfolio_request
)

# Pretty print the response from LUSID
prettyprint.portfolio_response(portfolio_response)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/problem+json', 'Date': 'Fri, 30 Sep 2022 15:49:12 GMT', 'Server': 'Kestrel', 'Transfer-Encoding': 'chunked', 'lusid-meta-success': 'False', 'lusid-meta-requestId': '0HML2VD5268R7:0000000D', 'lusid-meta-correlationId': '0HML2VD5268R7:0000000D', 'lusid-meta-duration': '759'})
HTTP response body: {"name":"UndefinedCurrencyFailure","errorDetails":[{"id":"CreateTransactionPortfolioRequest.BaseCurrency","detail":"Undefined currency: AUD"}],"code":407,"errors":{"CreateTransactionPortfolioRequest.BaseCurrency":["Undefined currency: AUD"]},"type":"https://docs.lusid.com/#section/Error-Codes/407","title":"The following currencies are undefined: AUD","status":400,"detail":"Please refer to the documentation on how to define a Currency","instance":"https://localhost.lusid.com/app/insights/logs/0HML2VD5268R7:0000000D","extensions":{}}


![Scopes](img/complexinstruments-portfolio.gif)

Now that you have a portfolio to hold the asset, you can create a transaction. You will create a 'StockIn' transaction which moves the asset into the portfolio.

*Run the cell below to create a transaction for the asset*

In [ ]:
take_on_transaction = models.TransactionRequest(
    transaction_id="39iw-090f9f9s",
    type="StockIn",
    instrument_identifiers={"Instrument/default/LusidInstrumentId": luid},
    transaction_date=(datetime.now(pytz.UTC) - timedelta(days=2)).isoformat(),
    settlement_date=datetime.now(pytz.UTC).isoformat(),
    units=1,
    transaction_price=models.TransactionPrice(price=18000000, type="Price"),
    total_consideration=models.CurrencyAndAmount(amount=18000000, currency="AUD"),
    source="Client",
    transaction_currency="AUD",
)

# Call LUSID to upsert our transactions
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=[take_on_transaction]
)

# Pretty print the response from LUSID
prettyprint.transactions_response(response, scope, portfolio_code)

You can now check the holdings of the portfolio to see if the asset has been added.

*Run the cell below to get the holdings of the portfolio to check that the asset has been added via the StockIn transaction*

In [ ]:
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope, code=portfolio_code, property_keys=["Instrument/default/Name"]
)

prettyprint.holdings_response(response, scope, portfolio_code)

Perhaps as part of the take on process of this asset, you have completed an independent valuation of it. You want to add the valuation to LUSID so that you can compare it with the cost price. You can do this using the quote store.

*Run the cell below to upsert a quote for the recent independent valuation*

In [ ]:
# Create a quote for the unit price of the base fund
instrument_quote = models.UpsertQuoteRequest(
    quote_id=models.QuoteId(
        quote_series_id=models.QuoteSeriesId(
            provider="DataScope",
            instrument_id=luid,
            instrument_id_type="LusidInstrumentId",
            quote_type="Price",
            field="Mid",
        ),
        effective_at=datetime.now(pytz.UTC).isoformat(),
    ),
    metric_value=models.MetricValue(value=18500000, unit="AUD"),
    lineage="InternalSystem",
)

# Call LUSID to upsert the quote
response = api_factory.build(lusid.api.QuotesApi).upsert_quotes(
    scope=scope, request_body={"price": instrument_quote}
)

prettyprint.upsert_quotes_response(response)

With this valuation stored as a quote in LUSID you can now aggregate across your portfolio to compare the cost of your asset with its current valuation. This is something that can be run manually or automated based on when the valuation is updated.

*Run the cell below to perform a valuation of your portfolio to compare the cost and value of your asset*

In [ ]:
recipe_code = "market_value"
recipe_scope = "private-assets"

inline_recipe = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    pricing=models.PricingContext(
        options=models.PricingOptions(
            allow_any_instruments_with_sec_uid_to_price_off_lookup=True
        )
    ),
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Quote.LusidInstrumentId.*",
                supplier="DataScope",
                data_scope=scope,
                quote_type="Price",
                field="Mid",
            ),
            models.MarketDataKeyRule(
                key="Fx.CurrencyPair.*",
                supplier="DataScope",
                data_scope=scope,
                quote_type="Rate",
                field="Mid",
            ),
        ],
        suppliers=models.MarketContextSuppliers(
            commodity="DataScope",
            credit="DataScope",
            equity="DataScope",
            fx="DataScope",
            rates="DataScope",
        ),
        options=models.MarketOptions(
            default_supplier="DataScope",
            default_instrument_code_type="LusidInstrumentId",
            default_scope=scope,
        ),
    ),
)

# Upsert recipe to LUSID
upsert_recipe_request = models.UpsertRecipeRequest(configuration_recipe=inline_recipe)
response = api_factory.build(
    lusid.api.ConfigurationRecipeApi
).upsert_configuration_recipe(upsert_recipe_request)

# Create the valuation request
valuation_request = models.ValuationRequest(
    recipe_id=models.ResourceId(scope=inline_recipe.scope, code=inline_recipe.code),
    metrics=[
        models.AggregateSpec(key="Instrument/default/LusidInstrumentId", op="Value"),
        models.AggregateSpec(key="Instrument/default/Name", op="Value"),
        models.AggregateSpec(key="Holding/default/Units", op="Sum"),
        models.AggregateSpec(key="Holding/default/Cost", op="Sum"),
        models.AggregateSpec(key="Valuation/PvInReportCcy", op="Sum"),
    ],
    valuation_schedule=models.ValuationSchedule(effective_at=datetime.now(pytz.UTC)),
    portfolio_entity_ids=[models.PortfolioEntityId(scope=scope, code=portfolio_code)],
    group_by=["Instrument/default/LusidInstrumentId"],
)

# Call LUSID to perform the valuation
response = api_factory.build(lusid.api.AggregationApi).get_valuation(
    valuation_request=valuation_request
)

# Pretty print the response
prettyprint.aggregation_response_generic(response)

Now you have created a bespoke instrument inside LUSID, created a custom schema to represent its important properties which can be extended to other similar assets, added it to a portfolio via a transaction and valued it.